# Amazon Bedrock API Wrapper Examples

This notebook demonstrates how to use the ezbedrock library to interact with Amazon Bedrock models.

## Setup

First, we import the necessary libraries and create a client with default parameters.
More information on Converse Parameters you can use: https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_Converse.html

In [1]:
from pydantic import BaseModel
from ezbedrock import BedrockClientWrapper

# Create a client with default parameters
bedrock = BedrockClientWrapper(
    model_id="anthropic.claude-v2",
    max_tokens=1000,  # Limits response length; set it high to avoid truncation issues
    temperature=0.7,  # Controls randomness (higher = more creative)
    top_p=0.95,  # Controls diversity of word choices
)

## Basic Text Example

The simplest way to get a text response from a model.

In [2]:
# Example of a basic text response from a model
print("Basic text example:")
response = bedrock.invoke_model("Tell me a joke")
print(response)

Basic text example:
Here's a silly joke: 

What do you call a dog magician? A labracadabrador!


## Example with Common Parameters

You can customize the model's behavior for each request using parameters like temperature.

In [3]:
# Example showing how to use common kwargs with invoke_model
print("Example with common kwargs:")
response = bedrock.invoke_model(
    prompt="Write a concise paragraph about artificial intelligence",
    system_prompt="You are an expert in artificial intelligence.",  # Optional system instruction
    temperature=0.2,  # Override just temperature for this less creative task
)
print(response)

Example with common kwargs:
Here is a concise paragraph about artificial intelligence:

Artificial intelligence (AI) refers to computer systems that can perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation. AI systems are powered by algorithms that learn from data and experience. Key focus areas in AI include machine learning, which allows systems to improve their performance by analyzing data; computer vision, which enables computers to interpret and understand visual inputs like images and videos; natural language processing, which focuses on enabling computers to communicate with people using everyday language; and robotics, where AI is applied to perform mechanical tasks with human-like dexterity. Though still an emerging field, AI has many practical applications and the potential to transform industries like healthcare, transportation, finance, and more. Ongoing research aims to make AI sy

## Structured Response with Pydantic

For more complex applications, you can enforce response structure using Pydantic models.

In [4]:
# Example showing Structured response with Pydantic
# Define a Pydantic model for structured responses
class JokeResponse(BaseModel):
    setup: str
    punchline: str
    category: str

print("Structured response with Pydantic:")
response = bedrock.invoke_model("Tell me a programming joke", response_model=JokeResponse)
print(response.model_dump_json(indent=2))

Structured response with Pydantic:
{
  "setup": "Why do Java developers wear glasses?",
  "punchline": "Because they can't C#.",
  "category": "programming"
}


## Generic JSON Response

When you need structured data but don't want to define a schema.

In [5]:
# Example showing a Generic JSON response
print("Generic JSON response:")
response = bedrock.invoke_model("List three programming languages with their key features", structured_output=True)
print(response)

Generic JSON response:
{'languages': [{'name': 'Python', 'features': ['Interpreted', 'Object-oriented', 'High-level', 'Dynamic typing', 'Extensive libraries']}, {'name': 'Java', 'features': ['Object-oriented', 'Statically typed', 'Portable', 'Robust', 'Secure']}, {'name': 'JavaScript', 'features': ['Interpreted', 'Object-oriented', 'Dynamic typing', 'First-class functions', 'Prototype-based']}]}


## Conversation with History

Create persistent conversations where the model remembers previous exchanges.

In [6]:
# Example showing a basic conversation
print("Conversation with history example:")
# Create a conversation
conversation = bedrock.create_conversation(system_prompt="You are a helpful assistant.")

# First message
response1 = conversation.send(
    "What are the 3 most popular programming languauges?", temperature=0.5
)
print("Response 1:")
print(response1)

response2 = conversation.send("Why is the first one so popular?")
print("Response 2:")
print(response2)

Conversation with history example:
Response 1:
Here are 3 of the most popular programming languages:

1. Python - Python is a high-level, general-purpose programming language that emphasizes code readability. It is very popular for web development, data analysis, artificial intelligence, and scientific computing. Python has a large and active community.

2. JavaScript - JavaScript is a scripting language used to create interactive effects within web browsers. It is an essential technology for front-end web development and is supported by all major browsers. JavaScript is also used in game development, mobile apps, server-side applications with Node.js, etc. 

3. Java - Java is a popular general-purpose programming language and development platform. It is fast, reliable, and secure. Java is commonly used for developing desktop applications, web applications, mobile apps, and enterprise software. Java runs on billions of devices worldwide. Its large community makes Java libraries and fra

## Advanced: Context Management

This demonstrates how the Conversation class uses a super simple technique of managing large conversation histories to:
1. Stay within token limits
2. Preserve context through summarization - Summaraizes the conversation when it gets over 4 messages long
3. Balance between complete history and recent exchanges

In [9]:
# Demonstrates how the conversation manages context when history gets large
print("Demonstrating context management:")

# Add multiple exchanges to demonstrate context management
for i in range(5):
    question = f"Question {i+1}: Tell me about another programming language"
    print(f"\nSending: {question}")
    response = conversation.send(question)
    print(f"Response summary: {response[:100]}...")

print("\nFinal history length:", len(conversation.history))
print("Full history length:", len(conversation.full_history))